In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tqdm

## Math for Logistic:
$$y \in \{-1,+1\}$$

$$h_{\theta}(x) = sigmoid(\theta^Tx) = \frac{1}{1+exp(-\theta^Tx)}$$

#### prob distribution for $y$

$$p(y|X;\theta) = h_{\theta}(yX)$$

#### MLE:

$$\max L(\theta) = \prod_{i=1}^{m}h_{\theta}(y^{(i)}x^{(i)})$$

#### Loss function:

$$\mathcal{l}(\theta) = \log L(\theta) = - \sum_{i=1}^{m}\log(1+\exp(-y^{(i)}\theta^Tx^{(i)}))$$

#### Gradient  of loss:

$$\frac{\partial}{\partial \theta}\mathcal l(\theta) = (1-h_{\theta}(yX)) \cdot yX$$

# Codes

In [2]:
sigmoid = lambda x: 1/(1+np.exp(-x))
d_sigmoid = lambda x: sigmoid(x)*(1-sigmoid(x))

In [1]:
class logisticRegression():
    def __init__(self, dim, alpha=1.0, zero_init=False):
        self.dim = dim+1
        self.alpha = alpha
        self.theta = np.zeros([dim+1]) if zero_init else np.random.randn(dim+1)
#         self.theta[0] = 0
        self.sigmoid = lambda x: 1/(1+np.exp(-x))
        self.h = lambda x: self.sigmoid(np.dot(self.theta,x))
    
    def fit(self, X, y, params=dict()):
        if X.shape[1]!=len(y):
            X = X.transpose()
        # add intercept
        X = np.insert(X, 0, values=1, axis=0)
        yX = y*X
        self.optim = self.set_params('SGD', 'optimizer', params)
        if self.optim == 'SGD':
            self.SGD(X,y,yX,params)
        elif self.optim == 'GD':
            self.GD(X,y,yX,params)    
        elif self.optim == 'SLGD':
            self.SLGD(X,y,yX,params)
        else:
            raise NameError(self.optim)
        
        
    def get_acc(self, X, y):
        return ((self.predict(X) * y)>0).sum()/len(y)
    
    def set_params(self, default, label, params):
        return default if label not in params.keys() else params[label]
        
    def GD(self,X, y, yX, params):
        rounds = self.set_params(50, 'rounds', params)
        decay = self.set_params(1.0, 'decay', params)
        L2 = self.set_params(0.5, 'l2_ratio', params)
        
        rnd = tqdm.trange(rounds, desc='Logisitic GD')            
        for i in rnd:
            dloss =  np.dot((self.h(yX)-1),yX.transpose())
            dloss if L2 == 0 else dloss + L2 * self.theta
            self.theta = self.theta - self.alpha * dloss
            self.acc = self.get_acc(X,y)
            rnd.set_description('Logistic GD (acc=%g)' % self.acc)
            self.alpha *= decay
        
    
    def SGD(self, X, y, yX, params):
        rounds = self.set_params(50, 'rounds', params)
        batchsize = self.set_params(32, 'batchsize', params)
        decay = self.set_params(1.0, 'decay', params)
        L2 = self.set_params(0.5, 'l2_ratio', params)
        
        num_item = yX.shape[1]
        num_batch = num_item//batchsize
        left_batch = num_item%batchsize
        num_batch = num_batch if left_batch == 0 else num_batch+1
        rnd = tqdm.trange(rounds, desc='Logisitic SGD')
        for i in rnd:
            shuffle = np.random.permutation(np.arange(num_item))
            for b in range(num_batch):
                if b!=num_batch-1:
                    batch = yX[:,shuffle[b*batchsize:(b+1)*batchsize]]
                else:
                    batch = yX[:,shuffle[b*batchsize:]]
                dloss = np.dot((self.h(batch)-1),batch.transpose())
                dloss = dloss if L2 == 0 else dloss + L2 * self.theta
                self.theta = self.theta - self.alpha * dloss
                
            self.acc = self.get_acc(X,y)
            rnd.set_description('Logistic SGD (acc=%g)' % self.acc)
            self.alpha *= decay    
    
    def SLGD(self, X, y, yX, params):
        rounds = self.set_params(50, 'rounds', params)
        batchsize = self.set_params(32, 'batchsize', params)
        decay = self.set_params(0.99, 'decay', params)
        
        num_item = yX.shape[1]
        num_batch = num_item//batchsize
        left_batch = num_item%batchsize
        num_batch = num_batch if left_batch == 0 else num_batch+1
        rnd = tqdm.trange(rounds, desc='Logisitic SLGD')
        for i in rnd:
            shuffle = np.random.permutation(np.arange(num_item))
            for b in range(num_batch):
                if b!=num_batch-1:
                    batch = yX[:,shuffle[b*batchsize:(b+1)*batchsize]]
                else:
                    batch = yX[:,shuffle[b*batchsize:]]
                dloss = np.dot((self.h(batch)-1),batch.transpose())
                # variance for Langevin Noise term is sqrt(lr)
                dloss = dloss + np.sqrt(self.alpha) * np.random.randn(self.dim)
                self.theta = self.theta - self.alpha * dloss
                
            self.acc = self.get_acc(X,y)
            rnd.set_description('Logistic SLGD (acc=%g)' % self.acc)
            self.alpha *= decay    
    
    
    def predict(self, X):
        if X.shape[0]!=self.dim and X.shape[0]!=self.dim-1:
            X = X.transpose()
        if X.shape[0]==self.dim-1:
            X = np.insert(X, 0, values=1, axis=0)            
        pred = self.h(X)
        pred[pred<0.5] = -1
        pred[pred>=0.5] = 1
        return pred.astype(int)
    
    def get_score(self, X):
        if X.shape[0]!=self.dim and X.shape[0]!=self.dim-1:
            X = X.transpose()
        if X.shape[0]==self.dim-1:
            X = np.insert(X, 0, values=1, axis=0)            
        pred = self.h(X)
        return pred

### Note & Reference:

For the Langevin Dynamics Optimization Strategy, I refered the blog 
https://henripal.github.io/blog/langevin and a ICML 2011 paper, 
[Bayesian Learning via Stochastic Gradient Langevin Dynamics](https://www.ics.uci.edu/~welling/publications/papers/stoclangevin_v6.pdf)


# Testing

In [2]:
from sklearn.datasets import load_breast_cancer

In [3]:
db = load_breast_cancer()

In [4]:
db.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [5]:
y = db['target']
X = db['data']

In [6]:
db.data.shape

(569, 30)

In [7]:
y[y==0] = -1

In [10]:
M = logisticRegression(30,1.)

params = {
    'rounds':2000,
    'optimizer':'SGD', 
    'l2_ratio':0.1, 
    'decay':0.99,
    'batchsize':32,
}

M.fit(X,y, params)

Logistic SGD (acc=0.922671): 100%|██████████| 2000/2000 [00:02<00:00, 749.57it/s]


In [33]:
M = logisticRegression(30,1.)

params = {
    'rounds':2000,
    'optimizer':'SLGD', 
    'decay':0.999,
    'batchsize':32,
}

M.fit(X,y, params)

Logistic SLGD (acc=0.927944): 100%|██████████| 2000/2000 [00:02<00:00, 806.41it/s]


In [11]:
print(M.get_score(X)[40:60])
print(M.predict(X)[40:60])
print(y[40:60])

[0.000000e+000 1.000000e+000 0.000000e+000 0.000000e+000 1.000000e+000
 0.000000e+000 1.000000e+000 4.323322e-245 1.000000e+000 1.000000e+000
 1.000000e+000 1.000000e+000 1.000000e+000 0.000000e+000 0.000000e+000
 1.000000e+000 0.000000e+000 0.000000e+000 1.000000e+000 1.000000e+000]
[-1  1 -1 -1  1 -1  1 -1  1  1  1  1  1 -1 -1  1 -1 -1  1  1]
[-1 -1 -1 -1 -1 -1  1 -1  1  1  1  1  1 -1 -1  1 -1 -1  1  1]


In [44]:
from sklearn.linear_model import LogisticRegression

In [47]:
clf = LogisticRegression().fit(X.transpose(),y)

/usr/share/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [49]:
(clf.predict(X.transpose())==y).sum()/len(y)

0.9595782073813708

In [6]:
a = np.random.randn(2,4)

In [18]:
c = np.insert(a, 0, values=1, axis=0)

In [19]:
c

array([[ 1.        ,  1.        ,  1.        ,  1.        ],
       [ 0.47978646,  1.06847799,  2.11185544, -0.18527605],
       [-0.49934084,  0.68689142,  0.08187441, -0.53404991]])